In [3]:
import numpy as np
import cv2 as cv
import sys
import getopt
import glob
from multiprocessing.dummy import Pool as ThreadPool
from common import splitfn
import os

# global settings
square_size = 35.15
threads_num = 8
subpix_windows_size = (5, 5)
term = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_COUNT, 1000, 1e-6)

dataset_dir = './mav_datasets/calibration_frontcam/img/'

img_dir_0 = './mav_datasets/calibration_frontcam/img_selected_0/'
img_dir_1 = './mav_datasets/calibration_frontcam/img_selected_1/'

debug_dir_0 = './mav_datasets/calibration_frontcam/debug_0/'
debug_dir_1 = './mav_datasets/calibration_frontcam/debug_1/'

undistort_dir_0 = './mav_datasets/calibration_frontcam/undistort_0/'
undistort_dir_1 = './mav_datasets/calibration_frontcam/undistort_1/'


In [4]:
# Arrays to store object points and image points from all the images.
pattern_size = (9, 6)

obj_points = []
img_points = []
images = glob.glob(img_dir_0 + '*.jpg')
os.system('rm -rf ' + debug_dir_0)
os.system('mkdir ' + debug_dir_0)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0) * square_size
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

# find chessboard function
def processImg(fn):
    img = cv.imread(fn, cv.IMREAD_GRAYSCALE)
    found, corners = cv.findChessboardCorners(img, pattern_size)
    if found:
        corners_subpix = cv.cornerSubPix(img, corners, subpix_windows_size, (-1, -1), term)

        if debug_dir_0:
            vis = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
            cv.drawChessboardCorners(vis, pattern_size, corners_subpix, found)
            _path, name, _ext = splitfn(fn)
            outfile = os.path.join(debug_dir_0, name + '.jpg')
            cv.imwrite(outfile, vis)

        return (corners_subpix.reshape(-1, 2), pattern_points)
        
    if not found:
        return None

if threads_num <= 1:
    chessboards = [processImg(fn) for fn in images]
else:
    pool = ThreadPool(threads_num)
    chessboards = pool.map(processImg, images)

detected = [x for x in chessboards if x is not None]
for (corners, pattern_points) in detected:
    img_points.append(corners)
    obj_points.append(pattern_points)

In [5]:
# Arrays to store object points and image points from all the images.
pattern_size = (6, 9)

obj_points = []
img_points = []
images = glob.glob(img_dir_1 + '*.jpg')
os.system('rm -rf ' + debug_dir_1)
os.system('mkdir ' + debug_dir_1)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0) * square_size
pattern_points = np.zeros((np.prod(pattern_size), 3), np.float32)
pattern_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

# find chessboard function
def processImg(fn):
    img = cv.imread(fn, cv.IMREAD_GRAYSCALE)
    found, corners = cv.findChessboardCorners(img, pattern_size)
    if found:
        corners_subpix = cv.cornerSubPix(img, corners, subpix_windows_size, (-1, -1), term)

        if debug_dir_1:
            vis = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
            cv.drawChessboardCorners(vis, pattern_size, corners_subpix, found)
            _path, name, _ext = splitfn(fn)
            outfile = os.path.join(debug_dir_1, name + '.jpg')
            cv.imwrite(outfile, vis)

        return (corners_subpix.reshape(-1, 2), pattern_points)
        
    if not found:
        return None

if threads_num <= 1:
    chessboards = [processImg(fn) for fn in images]
else:
    pool = ThreadPool(threads_num)
    chessboards = pool.map(processImg, images)

detected = [x for x in chessboards if x is not None]
for (corners, pattern_points) in detected:
    img_points.append(corners)
    obj_points.append(pattern_points)

In [6]:
# calculate camera distortion with all available images
print("calibrating, will take hours if there are too many points")
h, w = cv.imread(images[0], cv.IMREAD_GRAYSCALE).shape[:2]
rms, camera_matrix_1, distortion_1, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, (w, h), None, None)

print("\nRMS:", rms)
print("camera matrix:\n", camera_matrix_1)
print("distortion coefficients: ", distortion_1.ravel())

calibrating, will take hours if there are too many points

RMS: 0.3647045428982986
camera matrix:
 [[322.268871     0.         265.20837468]
 [  0.         323.35398364 210.86532454]
 [  0.           0.           1.        ]]
distortion coefficients:  [-3.42684145e-01  1.44269542e-01  7.41907658e-04  1.02642242e-04
 -3.22814428e-02]


In [7]:
RMS_0 = 0.28
camera_matrix_0 = np.array([
    [323.304409,     0.        , 264.18111025],
    [  0.      ,   323.7376457 , 213.45697442],
    [  0.      ,     0.        ,   1.        ]
])
distortion_0 = np.array([-3.42172150e-01,  1.42619049e-01,  6.17693970e-05,  1.79552140e-04, -3.15633945e-02])


In [8]:
# use opencv undistort function 0
os.system('rm -rf ' + undistort_dir_0)
os.system('mkdir ' + undistort_dir_0)

print(camera_matrix_0)
print(distortion_0)

images = glob.glob(dataset_dir + '*.jpg')
for fn in images if dataset_dir else []:
    _path, name, _ext = splitfn(fn)
    img_found = os.path.join(dataset_dir, name + '.jpg')
    outfile = os.path.join(undistort_dir_0, name + '.jpg')

    img = cv.imread(img_found)
    if img is None:
        continue

    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(camera_matrix_0, distortion_0, (w, h), 1, (w, h))
    dst = cv.undistort(img, camera_matrix_0, distortion_0, None, newcameramtx)

    # crop and save the image
    x, y, w, h = roi
    # dst = dst[y:y+h, x:x+w]

    cv.imwrite(outfile, dst)

[[323.304409     0.         264.18111025]
 [  0.         323.7376457  213.45697442]
 [  0.           0.           1.        ]]
[-3.42172150e-01  1.42619049e-01  6.17693970e-05  1.79552140e-04
 -3.15633945e-02]


In [9]:
# use opencv undistort function 0
os.system('rm -rf ' + undistort_dir_1)
os.system('mkdir ' + undistort_dir_1)

print(camera_matrix_1)
print(distortion_1)

images = glob.glob(dataset_dir + '*.jpg')
for fn in images if dataset_dir else []:
    _path, name, _ext = splitfn(fn)
    img_found = os.path.join(dataset_dir, name + '.jpg')
    outfile = os.path.join(undistort_dir_1, name + '.jpg')

    img = cv.imread(img_found)
    if img is None:
        continue

    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(camera_matrix_1, distortion_1, (w, h), 1, (w, h))
    dst = cv.undistort(img, camera_matrix_1, distortion_1, None, newcameramtx)

    # crop and save the image
    x, y, w, h = roi
    # dst = dst[y:y+h, x:x+w]

    cv.imwrite(outfile, dst)

[[322.268871     0.         265.20837468]
 [  0.         323.35398364 210.86532454]
 [  0.           0.           1.        ]]
[[-3.42684145e-01  1.44269542e-01  7.41907658e-04  1.02642242e-04
  -3.22814428e-02]]
